# Minimum Vertex Cover
(see also [Wikipedia](https://en.wikipedia.org/wiki/Vertex_cover))

To be able to directly compare to the [Pennylane implementation](https://github.com/PennyLaneAI/pennylane/blob/master/pennylane/qaoa/cost.py), we employ the following cost function:

$$
    \hat C = -\frac 34 \sum_{(i, j) \in E(G)} (\hat Z_i \hat Z_j  +  \hat Z_i  +  \hat Z_j)  + \sum_{i \in V(G)} \hat Z_i,
$$

where $E(G)$ is the set of edges and $V(G)$ is the set of vertices of the graph $G$ (we have a global minus sign since we _maximize_ the cost function).

In [ ]:
using DrWatson
@quickactivate "QAOA.jl"

include("../src/QAOA.jl")

using PyPlot
PyPlot.plt.style.use("paper.mplstyle")
using PyCall
np = pyimport("numpy")
nx = pyimport("networkx");

The following graph should be the same as [in this example](https://pennylane.ai/qml/demos/tutorial_qaoa_intro.html):

In [ ]:
N = 4
graph = nx.gnp_random_graph(N, 0.5, seed=7) 

figure(figsize=(3, 2))
nx.draw(graph, with_labels=true)

In [ ]:
h = -ones(N)
J = zeros(N, N)

Note that we have to __shift the edges by 1__ when going from Python to Julia:

In [ ]:
for edge in graph.edges
    h[edge[1] + 1] += 3/4.
    h[edge[2] + 1] += 3/4.
    J[(edge .+ (1, 1))...] = 3/4.
end
println(h)
J

Since our algorithm _maximizes_ the cost function, we put in __extra minus signs__ for the problem parameters.

In [ ]:
p = 2
problem QAOA.Problem(p, -h, -J)

__Using the wrapper function:__

In [ ]:
mvc_problem = QAOA.min_vertex_cover(graph, num_layers=p)

__Gradient optimization with [Zygote](https://fluxml.ai/Zygote.jl/latest/):__

In [ ]:
learning_rate = 0.01
cost, params, probs = QAOA.optimize_parameters(mvc_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)

__Optimization with [NLopt](https://nlopt.readthedocs.io/en/latest/):__

In [ ]:
cost, params, probs = QAOA.optimize_parameters(mvc_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]), :LN_COBYLA)

In [ ]:
figure(figsize=(4, 3))
bar(0:2^N-1, probs)
tight_layout()

The states `5` $\equiv |1010\rangle$ and `6` $\equiv |0110\rangle$ are indeed the correct solutions! Observe that `Yao.jl` uses the _opposite_ order when decoding binaries compared to, e.g., Pennylane (i.e. our solution `5` is their solution `10`).

### Other drivers

In [ ]:
mvc_problem_XX = QAOA.min_vertex_cover(graph, num_layers=p, driver=[X, X])
QAOA.optimize_parameters(mvc_problem_XX, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)

In [ ]:
mvc_problem_XX_YY = QAOA.min_vertex_cover(graph, num_layers=p, driver=[[X, X], [Y, Y]])
QAOA.optimize_parameters(mvc_problem_XX_YY, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)